# CHAPTER 9 - Quantum Support Vector Machines - Code

*Note*: You may skip the following four cells if you have alredy installed the right versions of all the libraries mentioned in *Appendix D*. This will likely NOT be the case if you are running this notebook on a cloud service such as Google Colab.

In [ ]:
pip install scikit-learn==1.2.1

In [ ]:
pip install pennylane==0.26

In [ ]:
pip install qiskit==0.39.2

In [ ]:
pip install qiskit_machine_learning==0.5.0

In [ ]:
import numpy as np

seed = 1234
np.random.seed(seed)

In [ ]:
from sklearn.datasets import load_wine

x,y = load_wine(return_X_y = True)

In [ ]:
x = x[:59+71]
y = y[:59+71]

In [ ]:
from sklearn.model_selection import train_test_split

x_tr, x_test, y_tr, y_test = train_test_split(x, y, train_size = 0.9)

In [ ]:
from sklearn.preprocessing import MaxAbsScaler

scaler = MaxAbsScaler()
x_tr = scaler.fit_transform(x_tr)

In [ ]:
x_test = scaler.transform(x_test)
x_test = np.clip(x_test,0,1)

In [ ]:
import pennylane as qml

nqubits = 4
dev = qml.device("lightning.qubit", wires = nqubits)

@qml.qnode(dev)
def kernel_circ(a, b):
    qml.AmplitudeEmbedding(
        a, wires=range(nqubits), pad_with=0, normalize=True)
    qml.adjoint(qml.AmplitudeEmbedding(
        b, wires=range(nqubits), pad_with=0, normalize=True))
    return qml.probs(wires = range(nqubits))

In [ ]:
from sklearn.svm import SVC
def qkernel(A, B):
    return np.array([[kernel_circ(a, b)[0] for b in B] for a in A])

svm = SVC(kernel = qkernel).fit(x_tr, y_tr)

In [ ]:
from sklearn.metrics import accuracy_score

print(accuracy_score(svm.predict(x_test), y_test))

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 8)

xs_tr = pca.fit_transform(x_tr)
xs_test = pca.transform(x_test)

In [ ]:
nqubits = 8
dev = qml.device("lightning.qubit", wires=nqubits)

@qml.qnode(dev)
def kernel_circ(a, b):
    qml.AngleEmbedding(a, wires=range(nqubits))
    qml.adjoint(qml.AngleEmbedding(b, wires=range(nqubits)))
    return qml.probs(wires = range(nqubits))

In [ ]:
svm = SVC(kernel = qkernel).fit(xs_tr, y_tr)

print(accuracy_score(svm.predict(xs_test), y_test))

In [ ]:
from itertools import combinations

def ZZFeatureMap(nqubits, data):

    # Number of variables that we will load:
    # could be smaller than the number of qubits.
    nload = min(len(data), nqubits) 

    for i in range(nload):
        qml.Hadamard(i)
        qml.RZ(2.0 * data[i], wires = i)

    for pair in list(combinations(range(nload), 2)):
        q0 = pair[0]
        q1 = pair[1]

        qml.CZ(wires = [q0, q1])
        qml.RZ(2.0 * (np.pi - data[q0]) *
            (np.pi - data[q1]), wires = q1)
        qml.CZ(wires = [q0, q1])

In [ ]:
nqubits = 4
dev = qml.device("lightning.qubit", wires = nqubits)

@qml.qnode(dev)
def kernel_circ(a, b):
    ZZFeatureMap(nqubits, a)
    qml.adjoint(ZZFeatureMap)(nqubits, b)
    return qml.probs(wires = range(nqubits))

svm = SVC(kernel=qkernel).fit(xs_tr, y_tr)
print(accuracy_score(svm.predict(xs_test), y_test))

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
from qiskit import *

In [ ]:
from qiskit.circuit import Parameter
parameter = Parameter("x")
qc = QuantumCircuit(1)
qc.rx(parameter, 0)

In [ ]:
from qiskit.circuit import ParameterVector
parameter = ParameterVector("x", length = 2)
qc = QuantumCircuit(2)
qc.rx(parameter[0], 0)
qc.rx(parameter[1], 1)

In [ ]:
from qiskit.circuit.library import ZZFeatureMap

zzfm = ZZFeatureMap(8)

In [ ]:
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit.providers.aer import AerSimulator

qkernel = QuantumKernel(feature_map = zzfm, 
            quantum_instance = AerSimulator())

In [ ]:
from qiskit_machine_learning.algorithms import QSVC

qsvm = QSVC(quantum_kernel = qkernel)
qsvm.fit(xs_tr, y_tr)

In [ ]:
print(accuracy_score(qsvm.predict(xs_test), y_test))

*Note*: In the following cell, you need to replace "1234" with your actual IBM token. Refer to *Appendix D* in the book for instructions on how to create an account and get your token. Be very careful not to disclose your token to anyone!

In [ ]:
ibm_token = "1234"
IBMQ.save_account(ibm_token)

In [ ]:
provider = IBMQ.load_account()

In [ ]:
from qiskit.providers.ibmq import *

dev_list = provider.backends(
    filters = lambda x: x.configuration().n_qubits >= 7,
    simulator = False)
dev = least_busy(dev_list)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 7)

xss_tr = pca.fit_transform(x_tr)

xss_test = pca.transform(x_test)

In [ ]:
zzfm = ZZFeatureMap(7)
qkernel = QuantumKernel(feature_map = zzfm, quantum_instance = dev)
qsvm = QSVC(quantum_kernel = qkernel)
qsvm.fit(xss_tr, y_tr)